In [4]:
import geopandas as gpd
import numpy as np
import pandas as pd
import os

In [5]:
class Beat_Data():

    def __init__(self):
        # Data Sources
        self.beat_list = np.arange(1, 278)
        d = {'beat_no': self.beat_list}
        self.assembly = pd.DataFrame(data=d)

        self.homicide_data = pd.read_csv('Data/CPD/Sep_01_2019_homicide_victims.csv')
        self.shooting_data = pd.read_csv('Data/CPD/Sep_01_2019_shooting_victims_details.csv')
        self.outreach_data = pd.read_csv('Data/Staffing/staff.csv')
        self.census_data = gpd.read_file('Data/Census/Beats/agg_aw_areal_Beats.geojson')
        self.edge_data = pd.read_csv('Data/edges.csv')

        self.beat_master = gpd.read_file('Data/Geo/Beats/geo_export_79ad3484-6fe3-40c7-bfae-0247adca4e23.shp')
        # creates a column called 'beat_no' in beat_master that gets beat_list as its series' values
        self.beat_master['beat_no'] = self.beat_list 
        self.community_data = self.Communities(self.outreach_data)
        self.outreach_data = self.Communities(self.outreach_data).outreach_assembly()

        columns = ['Community', 'beat_no']
        self.df_beats = pd.DataFrame(columns=columns)

    def homicide_assembly(self):
        # gets rid of excess data before I perform joins
        beat_filter = self.beat_master[['beat_no', 'beat_num', 'geometry']].copy()
        my_homicideData = self.homicide_data[['HOMICIDE_NO', 'BEAT_CD']].copy()
        # allows the same key to be present in both dataframes
        my_homicideData = my_homicideData.rename(columns={"BEAT_CD":"beat_num"})
        # replaces all the NaN's with 0's to avoid an error when I set the type
        my_homicideData = my_homicideData.fillna(0)
        # sets both the keys to the same datatype before I join them 
        beat_filter["beat_num"] = beat_filter["beat_num"].astype(int) 
        my_homicideData["beat_num"] = my_homicideData["beat_num"].astype(int)
        # left joins can work either way...I think. 
        hom_per_beat = beat_filter.merge(my_homicideData, on='beat_num', how='left')
        # groups unique values of homicide_IDs by their beat, but accidentally removes the geometries. 
        hom_per_beat = hom_per_beat.groupby(by='beat_no', as_index=False).agg({'HOMICIDE_NO': pd.Series.nunique})
        # creates a just beat-to-geometry dataset from the base
        beat_geos = self.beat_master[['beat_no', 'geometry']].copy()
        # rejoins the homicides by their geometries 
        # needed to merge beat_geos to hom_per_beat because the result wasn't a geopandas
        hom_per_beat = beat_geos.merge(hom_per_beat, on='beat_no', how='left')
        # renamed column to make distinction easier
        hom_per_beat = hom_per_beat.rename(columns={'HOMICIDE_NO':'count_homicides'})
        return hom_per_beat 

In [6]:
bd = Beat_Data()
homicides_per_beat = bd.homicide_assembly()

FileNotFoundError: [Errno 2] File b'Data/CPD/Sep_01_2019_homicide_victims.csv' does not exist: b'Data/CPD/Sep_01_2019_homicide_victims.csv'